In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import findspark
findspark.init("/home/rajdeep/spark-3.5.0-bin-hadoop3")

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('ecommerce').getOrCreate()

23/11/20 21:43:44 WARN Utils: Your hostname, DESKTOP-CSFBOLK resolves to a loopback address: 127.0.1.1; using 172.19.12.103 instead (on interface eth0)
23/11/20 21:43:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 21:43:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/20 21:43:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/11/20 21:43:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/11/20 21:43:49 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [5]:
df = spark.read.csv('data/Ecommerce_Customers.csv',inferSchema=True, header=True)

In [6]:
for val in df.head(1)[0]:
    print(val)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'],
               outputCol="features")

In [10]:
df_1 = assembler.transform(df)

In [11]:
df_1 = df_1.select(['features','Yearly Amount Spent'])

In [12]:
df_1.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [13]:
train_df,test_df = df_1.randomSplit([0.7, 0.3])

In [14]:
test_df.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                152|
|   mean|  509.6456046594871|
| stddev|  80.26996423262842|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [15]:
from pyspark.ml.regression import LinearRegression

In [16]:
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

In [17]:
lr_model = lr.fit(train_df)

23/11/20 21:44:06 WARN Instrumentation: [6057a8e6] regParam is zero, which might cause numerical instability and overfitting.
23/11/20 21:44:06 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/20 21:44:06 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [18]:
test_result = lr_model.evaluate(test_df)

In [19]:
test_result.rootMeanSquaredError

10.165149079879086

In [20]:
test_result.r2

0.9838568602547507

In [21]:
unlabled_data  = test_df.select(['features'])

In [22]:
unlabled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.4925366965402...|
|[30.8794843441274...|
|[31.1239743499119...|
|[31.1280900496166...|
|[31.5702008293202...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.6098395733896...|
|[31.8745516945853...|
|[31.9096268275227...|
|[31.9365486184489...|
|[31.9673209478824...|
|[32.0444861274404...|
|[32.0478146331398...|
|[32.0542618511847...|
|[32.0705462209254...|
|[32.0775900443291...|
|[32.0883806304482...|
|[32.1164001209281...|
+--------------------+
only showing top 20 rows



In [23]:
predictions = lr_model.transform(unlabled_data)

In [24]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 398.3871259251928|
|[30.4925366965402...|287.05540449858336|
|[30.8794843441274...|494.08009188327287|
|[31.1239743499119...|508.57537746432763|
|[31.1280900496166...| 565.0556602123752|
|[31.5702008293202...| 563.8451015471028|
|[31.5761319713222...| 543.6030755357617|
|[31.6005122003032...| 461.1801302052336|
|[31.6098395733896...| 427.2580746912158|
|[31.8745516945853...|398.29897944709205|
|[31.9096268275227...| 552.4813591348325|
|[31.9365486184489...|441.13368689976573|
|[31.9673209478824...|450.62426438604166|
|[32.0444861274404...|446.88497004870646|
|[32.0478146331398...| 480.7813529773048|
|[32.0542618511847...| 557.5003096262171|
|[32.0705462209254...| 532.4749690001247|
|[32.0775900443291...| 403.4539623768867|
|[32.0883806304482...| 518.5041571994971|
|[32.1164001209281...| 460.7363555559741|
+--------------------+------------